In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import csv
import random
from glob import glob
from tqdm import tqdm
from collections import namedtuple

import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch

# import sys
# # sys.path.append("../input/super-glue-pretrained-network")
# # from models.matching import Matching
# # from models.utils import (compute_pose_error, compute_epipolar_error,
# #                           estimate_pose, make_matching_plot,
# #                           error_colormap, AverageTimer, pose_auc, read_image,
# #                           rotate_intrinsics, rotate_pose_inplane,
# #                           scale_intrinsics)
from os import listdir
# import kornia as K
# import kornia.feature as KF
# import gc

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Importing LOFTR

In [4]:
dry_run = False
!pip install ../input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install ../input/kornia-loftr/kornia_moons-0.1.9-py3-none-any.whl

Processing /kaggle/input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
  Attempting uninstall: kornia
    Found existing installation: kornia 0.6.11
    Uninstalling kornia-0.6.11:
      Successfully uninstalled kornia-0.6.11
Processing /kaggle/input/kornia-loftr/kornia_moons-0.1.9-py3-none-any.whl


In [5]:
import os
import numpy as np
import cv2
import csv
from glob import glob
import torch
import matplotlib.pyplot as plt
import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF
import gc

In [6]:
device = torch.device('cuda')
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("../input/kornia-loftr/loftr_outdoor.ckpt")['state_dict'])
matcher = matcher.to(device).eval()


### Importing Super Glue

In [7]:
import sys
sys.path.append("../input/super-glue-pretrained-network")
from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
resize = [-1, ]
resize_float = True

config = {
    "superpoint": {
        "nms_radius": 3,
        "keypoint_threshold": 0.005,
        "max_keypoints": 2048
    },
    "superglue": {
        "weights": "outdoor",
        "sinkhorn_iterations": 100,
        "match_threshold": 0.2,
    }
}
matching = Matching(config).eval().to(device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [9]:
src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]


def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])


def load_torch_image(fname, device):
    img = cv2.imread(fname)
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img.to(device)

### Main Code

In [10]:
def load_resized_image(fname, resize): # function t read an image at a particular scale(when filename is given)
    img = cv2.imread(fname)
    h, w, _ = img.shape
    scale = resize / max(h, w) 
    w = int(w * scale)
    h = int(h * scale)
    
    img = cv2.resize(img, (w, h))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = torch.from_numpy(img)[None][None]/ 255.
    
    return img.to(device), scale

def load_image1(fname):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = K.image_to_tensor(img, False).float() /255.
    return img.to(device)

def load_resized_image1(img, resize): # function t read an image at a particular scale(when image itself is given)
    img=img[0,0,:,:]
    h, w = img.shape
    img=img.cpu().numpy()
    scale = resize / max(h, w) 
    w_new = int(w * scale)
    h_new = int(h * scale)
    
    img = cv2.resize(img, (w_new, h_new))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = torch.from_numpy(img)[None][None]
    
    return img.to(device), [h_new/h,w_new/w]

def scale_to_resized(mkpts0, mkpts1, scale1,scale2):
    ### scale to original im size because we used max_image_size
    # first point
    mkpts0[:, 0] = mkpts0[:, 0] / scale1[0]
    mkpts0[:, 1] = mkpts0[:, 1] / scale1[1]    
    # second point
    mkpts1[:, 0] = mkpts1[:, 0] / scale2[0]
    mkpts1[:, 1] = mkpts1[:, 1] / scale2[1]
    
    return mkpts0, mkpts1


def get_loftr_match(img1,img2):
    input_dict = {"image0": (img1), 
              "image1": (img2)}

    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    a1 = correspondences['keypoints0'].cpu().numpy()
    a2 = correspondences['keypoints1'].cpu().numpy()
    return a1,a2


from sklearn.cluster import DBSCAN

def crop_image(image, matching_points, eps, min_samples=5):
    # Convert matching points to numpy array
    matching_points = np.array(matching_points)

    # Apply DBSCAN clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(matching_points)

    # Find the most dense cluster
    unique_labels, label_counts = np.unique(clustering.labels_, return_counts=True)
#     print(unique_labels)
#     print(label_counts)
    most_dense_cluster_label = unique_labels[np.argmax(label_counts)]
#     print(unique_labels)
    # Get the matching points in the most dense cluster
    cluster_points = matching_points[clustering.labels_ == most_dense_cluster_label]
#     print(cluster_points)
    # Calculate the minimum and maximum coordinates in the cluster
    min_x = int(np.min(cluster_points[:, 0]))
    max_x = int(np.max(cluster_points[:, 0]))
    min_y = int(np.min(cluster_points[:, 1]))
    max_y = int(np.max(cluster_points[:, 1]))
    # Crop the image based on the cluster coordinates
    cropped_image = image[0,0,int(min_y):int(max_y), int(min_x):int(max_x)]

    return cropped_image,min_x,min_y

def get_superglue(img1,img2):
    pred = matching({"image0": img1, "image1": img2})
    pred = {k: v[0].detach().cpu().numpy() for k, v in pred.items()}
    # nd1 = time.time()
    kpts1, kpts2 = pred["keypoints0"], pred["keypoints1"]
    matches, conf = pred["matches0"], pred["matching_scores0"]

    valid = matches > -1
    a1 = kpts1[valid]
    a2 = kpts2[matches[valid]]
    mconf = conf[valid]
    return a1,a2


    

### Pipeline

In [11]:
F_dict = {}
import time
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row
    # Load the images. # sizes of 840,1040,1280
    st = time.time()
    img11=load_image1(f'{src}/test_images/{batch_id}/{image_1_id}.png')
    img22 =load_image1(f'{src}/test_images/{batch_id}/{image_2_id}.png')
    img1,scale1 = load_resized_image1(img11, 840)
    img2,scale2 = load_resized_image1(img22, 840)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts1=mkpts11
    mkpts2=mkpts22
    
#     img1,scale1 = load_resized_image1(img11, 840)
#     img2,scale2 = load_resized_image1(img22, 840)
#     mkpts11,mkpts22=get_superglue(img1,img2)
#     mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
#     mkpts1=np.vstack((mkpts1,mkpts11))
#     mkpts2=np.vstack((mkpts2,mkpts22))

    
    img1,scale1 = load_resized_image1(img11, 1040)
    img2,scale2 = load_resized_image1(img22, 1040)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts1=np.vstack((mkpts1,mkpts11))
    mkpts2=np.vstack((mkpts2,mkpts22))

    img1,scale1 = load_resized_image1(img11, 1240)
    img2,scale2 = load_resized_image1(img22, 1240)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts1=np.vstack((mkpts1,mkpts11))
    mkpts2=np.vstack((mkpts2,mkpts22))
    
    
    c1,minx1,miny1=crop_image(img11, mkpts1, 20, min_samples=5)
    c2,minx2,miny2=crop_image(img22, mkpts2, 20, min_samples=5)
    c1=c1.reshape(1,1,c1.shape[0],c1.shape[1])
    c2=c2.reshape(1,1,c2.shape[0],c2.shape[1])

    img1,scale1=load_resized_image1(c1,840)
    img2,scale2=load_resized_image1(c2,840)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts11+=np.array([minx1,miny1])
    mkpts22+=np.array([minx2,miny2])
    mkpts1=mkpts11
    mkpts2=mkpts22
    
#     img1,scale1=load_resized_image1(c1,840)
#     img2,scale2=load_resized_image1(c2,840)
#     mkpts11,mkpts22=get_superglue(img1,img2)
#     mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
#     mkpts11+=np.array([minx1,miny1])
#     mkpts22+=np.array([minx2,miny2])
#     mkpts1=np.vstack((mkpts1,mkpts11))
#     mkpts2=np.vstack((mkpts2,mkpts22))
    
    img1,scale1=load_resized_image1(c1,1040)
    img2,scale2=load_resized_image1(c2,1040)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts11+=np.array([minx1,miny1])
    mkpts22+=np.array([minx2,miny2])
    mkpts1=np.vstack((mkpts1,mkpts11))
    mkpts2=np.vstack((mkpts2,mkpts22))
    
    img1,scale1=load_resized_image1(c1,1280)
    img2,scale2=load_resized_image1(c2,1280)
    mkpts11,mkpts22=get_superglue(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts11+=np.array([minx1,miny1])
    mkpts22+=np.array([minx2,miny2])
    mkpts1=np.vstack((mkpts1,mkpts11))
    mkpts2=np.vstack((mkpts2,mkpts22))
    
    
    img1,scale1 = load_resized_image1(img11, 840)
    img2,scale2 = load_resized_image1(img22, 840)
    mkpts11,mkpts22=get_loftr_match(img1,img2)
    mkpts11,mkpts22=scale_to_resized(mkpts11, mkpts22, scale1,scale2)
    mkpts1=np.vstack((mkpts1,mkpts11))
    mkpts2=np.vstack((mkpts2,mkpts22))

#     img1=load_image1(f'{src}/test_images/{batch_id}/{image_1_id}.png')
#     img2=load_image1(f'{src}/test_images/{batch_id}/{image_2_id}.png')
    img1=img11
    img2=img22
    

    
    if len(mkpts1) > 7:
        F, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, 0.25, 0.9999, 10000)
#         F, inlier_mask = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, ransacReprojThreshold=0.25, confidence=0.99999, maxIters=10000)
        inliers = inliers > 0
        assert F.shape == (3, 3), 'Malformed F?'
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
        continue
    
    nd = time.time() 
    print("Running time: ", nd - st, " s")
    if (i < 3):
#         print("Running time: ", nd - st, " s")
        gc.collect()
        draw_LAF_matches(
        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1)),

        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts2).view(1,-1, 2),
                                    torch.ones(mkpts2.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts2.shape[0]).view(1,-1, 1)),
        torch.arange(mkpts1.shape[0]).view(-1,1).repeat(1,2),
        K.tensor_to_image(img1),
        K.tensor_to_image(img2),
        inliers,
        draw_dict={'inlier_color': (0.2, 1, 0.2),
                   'tentative_color': None, 
                   'feature_color': (0.2, 0.5, 1), 'vertical': False})
       
    
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')
        

/opt/conda/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Running time:  5.669443845748901  s
Running time:  1.1662216186523438  s
Running time:  0.9997663497924805  s
